<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8b29d27ebc8b455b1e10c93a5ad2987ad2becba994608ef2104e699ecb20e564
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-16 12:14:10
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.42 C
-------------------
Today PnL: 48.99 K (0.35%)
Current PnL: -17.05 L (-11.47%)
CY Booked + Current PnL: -6.02 L (-4.05%)
-------------------
Total profit:  2.24 L
Total loss:  -19.29 L
-------------------
Total Booked + Current PnL: 20.96 L (17.22%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.76%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.95 L (58.35%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SBIN,863.00,-11.34,65.0,M-LC,4.24,219632.0,18152.0,9071.0,0.49,9.01,4.13,13.51,60.0,2.00,1.57,22.74,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.86,56.0,H-LC,11.04,172073.0,15896.0,10204.0,0.06,10.18,5.93,16.71,22.0,1.56,1.23,29.39,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,104.27,58.0,M-SC,4.42,88019.0,-12758.0,12851.0,-0.04,-12.66,14.60,0.09,245.0,-0.99,0.63,15.64,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.83,68.0,H-MC,5.67,114125.0,13621.0,17016.0,-0.67,13.55,14.91,30.49,79.0,0.80,0.82,50.35,MH,ATH,METALS
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202006.0,1868.0,19029.0,0.11,0.93,9.42,10.44,4.0,0.10,1.44,6.55,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-30.16,53.0,H-SC,17.33,199723.0,-231.0,97465.0,-0.02,-0.12,48.80,48.63,115.0,-0.00,1.43,11.42,XR,NTT,MISC
35,ICICIPRULI,790.00,-20.65,45.0,H-MC,2.22,136572.0,784.0,41969.0,-0.15,0.58,30.73,31.48,107.0,0.02,0.98,12.77,X40,ATH,INSURANCE
12,BANKINDIA,190.00,-30.43,60.0,H-MC,5.80,180515.0,707.0,111703.0,-0.53,0.39,61.88,62.52,88.0,0.01,1.29,34.61,XR,NTT,BANKS
52,PGHH,17973.08,-31.08,51.0,H-MC,4.39,199500.0,-1320.0,70104.0,0.38,-0.66,35.14,34.25,80.0,-0.02,1.43,4.51,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-17.33,36.0,H-LC,5.61,297946.0,-17896.0,130560.0,-0.99,-5.67,43.82,35.67,5.0,-0.14,2.13,7.40,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-12.80,47.0,H-LC,8.19,219956.0,-31812.0,85013.0,-0.12,-12.64,38.65,21.13,27.0,-0.37,1.57,18.31,X40,ATH,PAINTS
31,HCLTECH,1943.91,1.08,49.0,H-LC,10.02,230790.0,-11134.0,74407.0,0.24,-4.60,32.24,26.15,8.0,-0.15,1.65,9.06,X40,ATH,IT
20,CIPLA,1795.00,-20.12,50.0,H-LC,5.81,213282.0,8782.0,32632.0,0.55,4.29,15.30,20.25,10.0,0.27,1.52,13.40,X40N,BTT,PHARMA
81,UNITDSPR,1644.00,-13.52,50.0,H-LC,7.90,226868.0,-7276.0,55900.0,0.28,-3.11,24.64,20.77,86.0,-0.13,1.62,3.60,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.20,-2.60,63.0,H-LC,1.01,241718.0,-4433.0,83634.0,0.56,-1.80,34.60,32.18,16.0,-0.05,1.73,33.59,X200,ATH,IT
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202006.0,1868.0,19029.0,0.11,0.93,9.42,10.44,4.0,0.10,1.44,6.55,X40,NTT,FMCG
51,NESTLEIND,1377.00,-7.97,59.0,H-LC,4.15,283187.0,17761.0,39023.0,-0.19,6.69,13.78,21.40,11.0,0.46,2.02,14.26,XY25,NTT,FMCG
57,RELIANCE,1533.00,-12.54,55.0,H-LC,5.58,218603.0,8237.0,20549.0,0.14,3.92,9.40,13.68,37.0,0.40,1.56,21.09,XY25,NTT,REFINERIES
84,VBL,671.64,-17.33,36.0,H-LC,5.61,297946.0,-17896.0,130560.0,-0.99,-5.67,43.82,35.67,5.0,-0.14,2.13,7.40,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,65.29,71.0,H-SC,8.47,143256.0,-39519.0,150591.0,6.73,-21.62,105.12,60.77,125.0,-0.26,1.02,28.31,XR,NTT,JEWELLERY
22,COFFEEDAY,80.00,-39.64,56.0,L-SC,42.02,88320.0,-25229.0,65277.0,3.56,-22.22,73.91,35.27,268.0,-0.39,0.63,116.17,XR,NTT,HOTELS
77,TRIDENT,48.00,-5.57,63.0,M-SC,5.37,73458.0,-18796.0,45103.0,2.55,-20.37,61.40,28.51,224.0,-0.42,0.52,25.70,XR,NTT,TEXTILES
55,RAJOOENG,143.33,-34.33,44.0,H-SC,10.65,99660.0,-2840.0,43671.0,2.53,-2.77,43.82,39.83,114.0,-0.07,0.71,4.17,AR,ATH,MISC
10,BAJAJHFL,152.00,-9.61,62.0,H-MC,6.05,190519.0,-9987.0,58756.0,2.49,-4.98,30.84,24.33,90.0,-0.17,1.36,9.86,X40N,BTT,FINANCE


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SHALBY,327.00,1065.87,65.0,M-SC,2.56,158482.0,-22661.0,68131.0,-1.74,-12.51,42.99,25.10,235.0,-0.33,1.13,27.01,XY24,NTT,HEALTHCARE
76,TITAGARH,1548.00,-0.57,65.0,H-SC,4.18,163074.0,-28941.0,107825.0,-1.73,-15.07,66.12,41.08,158.0,-0.27,1.17,38.18,XY24,NTT,ENGINEERING
83,VALIANTORG,838.00,-321.46,51.0,H-SC,7.12,123080.0,-40525.0,161838.0,-1.56,-24.77,131.49,74.15,139.0,-0.25,0.88,57.77,XR,NTT,CHEMICALS
4,ANGELONE,3033.00,3.06,35.0,H-SC,9.27,182434.0,-8572.0,66278.0,-1.21,-4.49,36.33,30.21,157.0,-0.13,1.30,15.46,X40N,NTT,FINANCE
84,VBL,671.64,-17.33,36.0,H-LC,5.61,297946.0,-17896.0,130560.0,-0.99,-5.67,43.82,35.67,5.0,-0.14,2.13,7.40,X40N,ATH,FMCG


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,98.37,59.0,H-SC,12.45,125550.0,-16929.0,36447.0,-0.47,-11.88,29.03,13.70,163.0,-0.46,0.90,48.18,OX40N,NTT,BANKS
78,TTKPRESTIG,770.0,104.27,58.0,M-SC,4.42,88019.0,-12758.0,12851.0,-0.04,-12.66,14.60,0.09,245.0,-0.99,0.63,15.64,OX40N,NTT,DURABLES
19,CERA,9475.0,-20.05,46.0,H-SC,2.16,145832.0,-30071.0,72099.0,0.34,-17.10,49.44,23.89,149.0,-0.42,1.04,25.94,OX40N,NTT,CERAMICS
45,KANSAINER,340.0,-67.61,59.0,H-SC,1.73,223335.0,-46332.0,82656.0,0.81,-17.18,37.01,13.47,138.0,-0.56,1.60,12.96,XY24,NTT,PAINTS
73,TATAELXSI,9161.0,-20.18,52.0,H-MC,7.82,108053.0,-23782.0,66010.0,0.04,-18.04,61.09,32.03,98.0,-0.36,0.77,21.03,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-26.12,54.0,H-LC,12.82,294389.0,-51567.0,126411.0,0.64,-14.91,42.94,21.64,1.0,-0.41,2.10,4.29,X40,ATH,IT
40,INFY,2275.00,-17.80,52.0,H-LC,8.82,320693.0,7805.0,163874.0,-0.19,2.49,51.10,54.87,3.0,0.05,2.29,9.31,X40,BTT,IT
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202006.0,1868.0,19029.0,0.11,0.93,9.42,10.44,4.0,0.10,1.44,6.55,X40,NTT,FMCG
84,VBL,671.64,-17.33,36.0,H-LC,5.61,297946.0,-17896.0,130560.0,-0.99,-5.67,43.82,35.67,5.0,-0.14,2.13,7.40,X40N,ATH,FMCG
1,ABB,7934.00,-38.60,62.0,H-LC,11.13,256440.0,-5179.0,124399.0,0.07,-1.98,48.51,45.57,7.0,-0.04,1.83,9.12,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,65.0,M-MC,6.97,228819.0,3857.0,163789.0,0.07,1.71,71.58,74.53,192.0,0.02,1.63,34.52,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,11.93,60.0,H-MC,3.71,193180.0,28510.0,39467.0,2.26,17.31,20.43,41.28,89.0,0.72,1.38,26.68,X40N,ATH,AC
85,VOLTAS,1918.49,1.86,61.0,H-MC,2.67,213690.0,21948.0,74086.0,0.62,11.45,34.67,50.08,99.0,0.30,1.53,19.02,XY25,ATH,AC
30,HAVELLS,2069.16,1.33,65.0,H-MC,1.82,251160.0,3429.0,71631.0,0.93,1.38,28.52,30.30,92.0,0.05,1.79,14.86,X40,ATH,ELECTRICAL
24,DABUR,735.00,-3.30,56.0,H-MC,6.56,210405.0,13377.0,76251.0,-0.26,6.79,36.24,45.49,102.0,0.18,1.50,20.72,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.83,68.0,H-MC,5.67,114125.0,13621.0,17016.0,-0.67,13.55,14.91,30.49,79.0,0.80,0.82,50.35,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-39.02,60.0,M-SC,0.61,103050.0,11552.0,67199.0,2.26,12.62,65.21,86.07,223.0,0.17,0.74,49.65,XR,NTT,DURABLES
37,INDIAMART,4850.92,-50.72,52.0,H-SC,12.20,137795.0,14459.0,119303.0,-0.35,11.72,86.58,108.45,119.0,0.12,0.98,36.70,AR,ATH,MISC
87,WIPRO,420.00,-11.07,53.0,M-LC,5.60,155980.0,5035.0,104413.0,0.12,3.34,66.94,72.51,53.0,0.05,1.11,9.56,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,60.0,H-MC,5.80,180515.0,707.0,111703.0,-0.53,0.39,61.88,62.52,88.0,0.01,1.29,34.61,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,56.0,L-SC,42.02,88320.0,-25229.0,65277.0,3.56,-22.22,73.91,35.27,268.0,-0.39,0.63,116.17,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.17,48.0,H-SC,13.34,89118.0,-11845.0,106380.0,0.54,-11.73,119.37,93.63,148.0,-0.11,0.64,32.74,SR,ATH,CHEMICALS
49,MASFIN,398.61,-15.09,60.0,H-SC,12.94,96450.0,-1530.0,23129.0,0.50,-1.56,23.98,22.05,152.0,-0.07,0.69,40.20,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.83,68.0,H-MC,5.67,114125.0,13621.0,17016.0,-0.67,13.55,14.91,30.49,79.0,0.80,0.82,50.35,MH,ATH,METALS
83,VALIANTORG,838.00,-321.46,51.0,H-SC,7.12,123080.0,-40525.0,161838.0,-1.56,-24.77,131.49,74.15,139.0,-0.25,0.88,57.77,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,56.0,L-SC,42.02,88320.0,-25229.0,65277.0,3.56,-22.22,73.91,35.27,268.0,-0.39,0.63,116.17,XR,NTT,HOTELS
8,ATULAUTO,844.00,3905.41,65.0,M-SC,6.45,124797.0,-22571.0,74391.0,2.11,-15.32,59.61,35.16,236.0,-0.30,0.89,27.47,XY24,NTT,AUTO
61,SAMMAANCAP,326.00,-170.45,62.0,M-SC,2.62,81804.0,-20406.0,113798.0,0.39,-19.96,139.11,91.37,208.0,-0.18,0.58,32.79,XY25,NTT,FINANCE
82,VAIBHAVGBL,521.00,65.29,71.0,H-SC,8.47,143256.0,-39519.0,150591.0,6.73,-21.62,105.12,60.77,125.0,-0.26,1.02,28.31,XR,NTT,JEWELLERY
50,NATIONALUM,247.44,-42.83,68.0,H-MC,5.67,114125.0,13621.0,17016.0,-0.67,13.55,14.91,30.49,79.0,0.80,0.82,50.35,MH,ATH,METALS


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.46
1,25,42.74
2,50,73.14


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.25
LC    33.93
MC    22.79
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.48
X40      15.02
X40N     12.76
XR       11.39
XY25      9.63
AR        8.50
OX40N     8.05
X200      1.73
MH        1.70
X5K       1.44
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.37
H-LC    25.82
H-MC    19.88
M-SC    13.35
M-LC     7.06
M-MC     2.61
L-SC     1.53
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.13,-14.34,76.68
FMCG,11.61,-4.16,40.60
FINANCE,9.49,-15.80,61.15
BANKS,7.63,-12.27,63.96
MISC,7.36,-11.54,73.57
PAINTS,5.77,-14.40,31.46
ELECTRICAL,5.59,-5.06,62.97
AC,3.61,4.04,28.46
INSURANCE,3.36,-7.21,43.98


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2857203.0,22
XR,1314028.0,14
AR,1304480.0,9
X40,746799.0,10
X40N,625267.0,10
OX40N,520231.0,10
XY25,403913.0,7
SR,244283.0,2
X5K,116168.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3442547.0,29
M-SC,1375724.0,17
H-LC,1258954.0,15
H-MC,1171860.0,15
M-LC,389492.0,5
M-MC,312094.0,2
L-SC,245912.0,3
L-MC,60162.0,1
L-LC,37798.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1042805.0      6
M-SC       XY24       796876.0      7
H-SC       AR         784724.0      5
           XR         763141.0      7
H-MC       XY24       485394.0      4
H-LC       X40        468734.0      5
           AR         357524.0      2
M-MC       XY24       312094.0      2
H-SC       X40N       297748.0      4
           OX40N      248325.0      4
           SR         244283.0      2
H-LC       X40N       229296.0      4
H-MC       X40        209840.0      4
           XY25       183674.0      2
L-SC       XR         162307.0      2
M-SC       AR         162232.0      2
M-LC       XY24       159840.0      2
M-SC       OX40N      122291.0      4
M-LC       X5K        116168.0      1
M-SC       XY25       113798.0      1
           XR         112302.0      2
H-MC       XR         111703.0      1
M-LC       XR         104413.0      1
H-MC       X40N        98223.0      2
H-LC       X200        83634.0      1
L-SC       OX40N       83605.0      1
M-SC       X40         68225.0      1
H-MC       OX40N       66010.0      1
H-SC       MH          61521.0      1
H-LC       XY24        60194.0      1
L-MC       XR          60162.0      1
H-LC       XY25        59572.0      2
L-LC       XY25        37798.0      1
H-MC       MH          17016.0      1
M-LC       XY25         9071.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
